# Setting Workspace

In [ ]:
!pip install ipython-autotime
!pip install transformers

     |████████████████████████████████| 2.6 MB 8.5 MB/s 
     |████████████████████████████████| 3.3 MB 42.1 MB/s 
     |████████████████████████████████| 636 kB 21.6 MB/s 
     |████████████████████████████████| 895 kB 40.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
%load_ext autotime

time: 134 µs (started: 2021-07-24 05:18:42 +00:00)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 16.2 s (started: 2021-07-24 05:18:42 +00:00)


In [ ]:
!pwd
import os
os.chdir('/content/drive/My Drive/HLT/')
!pwd

/content
/content/drive/My Drive/HLT
time: 262 ms (started: 2021-07-24 05:18:59 +00:00)


# Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, pipeline, TFBertModel, TFGPT2Model, TFOpenAIGPTModel
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,roc_auc_score,roc_curve,r2_score,recall_score,confusion_matrix,precision_recall_curve
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from tensorflow.keras.layers import InputLayer, Embedding, Concatenate, TimeDistributed, Bidirectional, GRU
from tensorflow.keras.layers import LSTM, Dense,Flatten,Conv2D,Conv1D,GlobalMaxPooling1D,GlobalMaxPool1D
from sklearn.metrics import classification_report,roc_auc_score,roc_curve,r2_score,recall_score,confusion_matrix,precision_recall_curve
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout

time: 8.2 s (started: 2021-07-24 05:18:59 +00:00)


In [ ]:
pd.set_option('display.max_colwidth', -1) #show all text in col
pd.set_option('display.max_rows', None) #show all rows

time: 3.55 ms (started: 2021-07-24 05:19:07 +00:00)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


# Create GPT-2

In [ ]:
train_df = pd.read_csv("datasets/train.csv") 
train_df['headline']=train_df['headline'].apply(str)
train_df.head()

,headline,is_sarcastic
0,bill de blasio thinks hes proved haters wrong comes prek,0
1,baby doesnt realize white supremacist yet,1
2,spring sprung arctic way early,0
3,mayor daleys son appointed head illinois nepotist party,1
4,poll grandsons talented,1


time: 54.3 ms (started: 2021-07-23 20:09:47 +00:00)


In [ ]:
test_df = pd.read_csv("datasets/test.csv") 
test_df['headline']=test_df['headline'].apply(str)
test_df.head()

,headline,is_sarcastic
0,jessie james decker shares inspiring message postbaby bodies,0
1,frustrated russian officials struggling get policies dysfunctional trump administration,1
2,second saturday staten island art walk,0
3,chris christie emits loud sob paul ryan asks crowd whether worse years ago,1
4,chicago police department monitor interactions public using new bullet cams,1


time: 24.9 ms (started: 2021-07-23 20:09:47 +00:00)


In [ ]:
print("Train size:{}".format(train_df.shape))
print("Test size:{}".format(test_df.shape))

Train size:(22702, 2)
Test size:(4007, 2)
time: 2.41 ms (started: 2021-07-23 20:09:47 +00:00)


In [ ]:
train_y = train_df['is_sarcastic']
test_y = test_df['is_sarcastic']

train_x = train_df['headline']
test_x = test_df['headline']
train_x.shape,train_y.shape,test_x.shape,test_y.shape

((22702,), (22702,), (4007,), (4007,))

time: 6.12 ms (started: 2021-07-23 20:09:47 +00:00)


In [ ]:
def transformer_gpt_embedding(name,inp,model_name):

    model = model_name.from_pretrained(name)
    tokenizer = AutoTokenizer.from_pretrained(name)
    tokenizer.pad_token = "[PAD]"
    pipe = pipeline('feature-extraction', model=model, 
                tokenizer=tokenizer)
    features = pipe(inp)
    features = np.squeeze(features)
    return features

time: 3.99 ms (started: 2021-07-23 20:09:48 +00:00)


In [ ]:
'''
Embedding trainset

'''
train_x_list = train_x.tolist()
train_y_list = train_y.to_list()

time: 2.19 ms (started: 2021-07-23 20:09:48 +00:00)


In [ ]:
len(train_x_list)

22702

time: 3.13 ms (started: 2021-07-23 20:09:48 +00:00)


In [ ]:
len(train_y_list)

22702

time: 2.98 ms (started: 2021-07-23 20:09:48 +00:00)


In [ ]:
embedding_features=transformer_gpt_embedding('openai-gpt',train_x_list[16000:18000],TFGPT2Model)

You are using a model of type openai-gpt to instantiate a model of type gpt2. This is not supported for all configurations of models and can yield errors.
Some layers from the model checkpoint at openai-gpt were not used when initializing TFGPT2Model: ['transformer/positions_embed/embeddings:0', 'transformer/tokens_embed/weight:0']
- This IS expected if you are initializing TFGPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFGPT2Model were not initialized from the model checkpoint at openai-gpt and are newly initialized: ['transformer/wpe/embeddings:0', 'transformer/ln_f/gamma:0', 'transformer/wte/weight:0', 'tran

time: 1min 54s (started: 2021-07-23 20:09:48 +00:00)


In [ ]:
embedding_features.shape

(2000, 22, 768)

time: 39.1 ms (started: 2021-07-23 20:11:42 +00:00)


In [ ]:
train_gpt_2_x = embedding_features[:,0,:]

time: 815 µs (started: 2021-07-23 20:11:42 +00:00)


In [ ]:
train_gpt_2_x.shape

(2000, 768)

time: 4.19 ms (started: 2021-07-23 20:11:42 +00:00)


In [ ]:
train_gpt_2_x_df = pd.DataFrame(train_gpt_2_x)

time: 1.24 ms (started: 2021-07-23 20:11:42 +00:00)


In [ ]:
train_gpt_2_x_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,-0.002061,-0.002264,0.180387,0.342313,0.118273,0.130569,-0.116520,-0.057395,0.051470,0.323987,0.072375,0.045739,-0.035608,0.000557,-0.075084,-0.084931,0.503598,0.289106,0.129671,0.016092,-0.128839,0.172460,0.351176,0.331908,0.177274,0.118245,-0.384552,0.401597,-0.275888,0.120991,-0.223565,-0.226685,0.205774,0.008952,0.068484,-0.349223,-0.032744,0.107711,0.000655,0.496928,...,-0.266953,0.284062,-0.121872,-0.132264,0.399864,0.245557,0.273040,0.875272,0.054985,0.180182,-0.263273,0.208326,-0.254565,0.352763,-0.212451,0.305601,0.072043,0.115318,-0.076159,0.242629,-0.289004,0.216222,0.219496,0.599424,-0.013554,-0.072593,0.386422,-0.210291,-0.197253,-0.258634,0.160648,0.352057,-0.382958,-0.001053,0.119992,0.197172,-0.224918,-0.194209,-0.002806,0.088657
1,0.035570,-0.078747,0.166971,0.449375,0.135167,0.356245,0.006773,-0.221571,-0.004590,0.379338,0.007414,0.230455,0.104063,-0.292584,-0.186182,0.035756,0.463720,0.192056,0.049968,0.020097,-0.319611,0.218269,0.448210,0.100992,0.236918,-0.011066,-0.260592,0.202784,-0.276461,0.238261,-0.140779,-0.151388,0.233092,0.005814,0.206934,-0.302712,0.032799,0.038047,-0.093145,0.497442,...,-0.268633,0.200988,-0.209689,-0.355098,0.343173,0.040906,0.123836,0.728870,-0.067836,0.102496,-0.115386,0.031986,-0.062115,0.325076,-0.272212,0.150844,0.068174,-0.054550,-0.047557,0.151750,-0.225946,0.311375,0.360453,0.714059,0.032858,-0.093735,0.413167,-0.237622,-0.142015,-0.247524,0.252907,0.402336,-0.425228,-0.016664,0.116695,0.233497,-0.246594,-0.320861,0.069199,0.189032
2,0.063276,0.049074,0.089819,0.316524,0.162780,0.219830,-0.011014,-0.192138,-0.012340,0.360250,0.106049,0.101695,0.115894,-0.215686,-0.057649,-0.055260,0.354331,0.251998,0.115984,-0.056262,-0.112766,0.216191,0.339150,0.196060,0.145609,0.289920,-0.317934,0.231668,-0.124328,0.173057,-0.051350,-0.209719,0.510537,-0.036796,0.259837,-0.261062,0.064266,0.050542,-0.068799,0.349878,...,-0.305563,0.161505,-0.037811,-0.090168,0.309235,0.162671,0.206860,0.964671,-0.068371,0.117655,-0.017951,0.024181,-0.090213,0.351990,-0.178405,0.163905,0.000490,-0.029045,-0.068018,0.208254,-0.409466,0.255130,0.183905,0.641039,0.098808,-0.029857,0.293054,-0.199315,-0.114853,-0.304991,0.182638,0.293251,-0.376552,-0.098182,0.035056,0.218602,-0.213292,-0.050599,-0.123835,0.135782
3,-0.029102,0.013758,0.198493,0.477120,0.108808,0.201981,-0.194387,-0.060702,-0.031330,0.385378,0.136366,0.221585,0.061824,-0.031325,-0.081020,-0.197730,0.621946,0.138601,0.099390,0.059797,-0.020619,0.150545,0.429956,0.026791,0.117810,-0.001794,-0.255542,0.249620,-0.175645,0.128698,-0.134654,-0.177303,0.150077,-0.325769,0.182269,-0.248867,-0.026056,0.117584,-0.065510,0.406648,...,-0.270936,0.300115,-0.149585,-0.205368,0.252334,0.134237,0.372557,1.124579,-0.063513,0.134561,-0.068454,-0.016073,-0.222364,0.171299,-0.160047,0.066940,0.110626,-0.000953,-0.147314,0.084507,-0.458088,0.249273,0.122423,0.649907,0.108944,-0.076069,0.437852,-0.085238,-0.214961,-0.121516,0.163508,0.135172,-0.325937,0.029340,0.025050,0.277459,-0.112699,-0.101002,-0.244339,0.024031
4,0.091385,0.042672,0.114439,0.413912,-0.082305,0.378680,-0.148158,-0.181026,0.087492,0.425562,-0.029053,0.157339,0.055909,-0.121342,-0.042284,-0.043861,0.451341,0.336374,0.138938,0.057634,-0.128077,0.228529,0.467262,0.231662,0.274576,0.029675,-0.283174,0.304914,-0.110095,0.048199,-0.063913,-0.206986,0.315677,-0.102813,0.166746,-0.226050,-0.011948,0.189479,-0.075256,0.513800,...,-0.383319,0.177071,-0.063680,-0.206625,0.320533,0.256104,0.201519,1.250820,-0.018126,0.040443,-0.181061,0.191698,-0.278391,0.381524,-0.145215,0.228216,0.088060,0.012423,-0.043392,-0.006628,-0.530300,0.428057,0.198738,0.732277,0.042085,-0.032629,0.378909,-0.218073,-0.048241,-0.292797,0.113202,0.252028,-0.3

time: 156 ms (started: 2021-07-23 20:11:42 +00:00)


In [ ]:
train_gpt_2_x_df['is_sarcastic'] = train_y_list[16000:18000]

time: 2.77 ms (started: 2021-07-23 20:11:43 +00:00)


In [ ]:
train_gpt_2_x_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,is_sarcastic
0,-0.002061,-0.002264,0.180387,0.342313,0.118273,0.130569,-0.116520,-0.057395,0.051470,0.323987,0.072375,0.045739,-0.035608,0.000557,-0.075084,-0.084931,0.503598,0.289106,0.129671,0.016092,-0.128839,0.172460,0.351176,0.331908,0.177274,0.118245,-0.384552,0.401597,-0.275888,0.120991,-0.223565,-0.226685,0.205774,0.008952,0.068484,-0.349223,-0.032744,0.107711,0.000655,0.496928,...,0.284062,-0.121872,-0.132264,0.399864,0.245557,0.273040,0.875272,0.054985,0.180182,-0.263273,0.208326,-0.254565,0.352763,-0.212451,0.305601,0.072043,0.115318,-0.076159,0.242629,-0.289004,0.216222,0.219496,0.599424,-0.013554,-0.072593,0.386422,-0.210291,-0.197253,-0.258634,0.160648,0.352057,-0.382958,-0.001053,0.119992,0.197172,-0.224918,-0.194209,-0.002806,0.088657,1
1,0.035570,-0.078747,0.166971,0.449375,0.135167,0.356245,0.006773,-0.221571,-0.004590,0.379338,0.007414,0.230455,0.104063,-0.292584,-0.186182,0.035756,0.463720,0.192056,0.049968,0.020097,-0.319611,0.218269,0.448210,0.100992,0.236918,-0.011066,-0.260592,0.202784,-0.276461,0.238261,-0.140779,-0.151388,0.233092,0.005814,0.206934,-0.302712,0.032799,0.038047,-0.093145,0.497442,...,0.200988,-0.209689,-0.355098,0.343173,0.040906,0.123836,0.728870,-0.067836,0.102496,-0.115386,0.031986,-0.062115,0.325076,-0.272212,0.150844,0.068174,-0.054550,-0.047557,0.151750,-0.225946,0.311375,0.360453,0.714059,0.032858,-0.093735,0.413167,-0.237622,-0.142015,-0.247524,0.252907,0.402336,-0.425228,-0.016664,0.116695,0.233497,-0.246594,-0.320861,0.069199,0.189032,0
2,0.063276,0.049074,0.089819,0.316524,0.162780,0.219830,-0.011014,-0.192138,-0.012340,0.360250,0.106049,0.101695,0.115894,-0.215686,-0.057649,-0.055260,0.354331,0.251998,0.115984,-0.056262,-0.112766,0.216191,0.339150,0.196060,0.145609,0.289920,-0.317934,0.231668,-0.124328,0.173057,-0.051350,-0.209719,0.510537,-0.036796,0.259837,-0.261062,0.064266,0.050542,-0.068799,0.349878,...,0.161505,-0.037811,-0.090168,0.309235,0.162671,0.206860,0.964671,-0.068371,0.117655,-0.017951,0.024181,-0.090213,0.351990,-0.178405,0.163905,0.000490,-0.029045,-0.068018,0.208254,-0.409466,0.255130,0.183905,0.641039,0.098808,-0.029857,0.293054,-0.199315,-0.114853,-0.304991,0.182638,0.293251,-0.376552,-0.098182,0.035056,0.218602,-0.213292,-0.050599,-0.123835,0.135782,0
3,-0.029102,0.013758,0.198493,0.477120,0.108808,0.201981,-0.194387,-0.060702,-0.031330,0.385378,0.136366,0.221585,0.061824,-0.031325,-0.081020,-0.197730,0.621946,0.138601,0.099390,0.059797,-0.020619,0.150545,0.429956,0.026791,0.117810,-0.001794,-0.255542,0.249620,-0.175645,0.128698,-0.134654,-0.177303,0.150077,-0.325769,0.182269,-0.248867,-0.026056,0.117584,-0.065510,0.406648,...,0.300115,-0.149585,-0.205368,0.252334,0.134237,0.372557,1.124579,-0.063513,0.134561,-0.068454,-0.016073,-0.222364,0.171299,-0.160047,0.066940,0.110626,-0.000953,-0.147314,0.084507,-0.458088,0.249273,0.122423,0.649907,0.108944,-0.076069,0.437852,-0.085238,-0.214961,-0.121516,0.163508,0.135172,-0.325937,0.029340,0.025050,0.277459,-0.112699,-0.101002,-0.244339,0.024031,0
4,0.091385,0.042672,0.114439,0.413912,-0.082305,0.378680,-0.148158,-0.181026,0.087492,0.425562,-0.029053,0.157339,0.055909,-0.121342,-0.042284,-0.043861,0.451341,0.336374,0.138938,0.057634,-0.128077,0.228529,0.467262,0.231662,0.274576,0.029675,-0.283174,0.304914,-0.110095,0.048199,-0.063913,-0.206986,0.315677,-0.102813,0.166746,-0.226050,-0.011948,0.189479,-0.075256,0.513800,...,0.177071,-0.063680,-0.206625,0.320533,0.256104,0.201519,1.250820,-0.018126,0.040443,-0.181061,0.191698,-0.278391,0.381524,-0.145215,0.228216,0.088060,0.012423,-0.043392,-0.006628,-0.530300,0.428057,0.198738,0.732277,0.042085,-0.032629,0.378909,-0.218073,-0.048241,-0.292797,0.113202,0.252028,-0.302155,-0.042468,0.039976,0.262477

time: 108 ms (started: 2021-07-23 20:11:43 +00:00)


In [ ]:
train_gpt_2_x_df.to_csv('datasets/train_gpt_2_16000_18000.csv', encoding='utf-8', index=False)

time: 2.25 s (started: 2021-07-23 20:11:43 +00:00)


In [ ]:
'''
Embedding testset

'''
test_x_list = test_x.tolist()
test_y_list = test_y.to_list()

time: 1.74 ms (started: 2021-07-23 19:52:47 +00:00)


In [ ]:
len(test_x_list)

4007

time: 3.47 ms (started: 2021-07-23 19:52:49 +00:00)


In [ ]:
len(test_y_list)

4007

time: 3.07 ms (started: 2021-07-23 20:07:17 +00:00)


In [ ]:
embedding_features=transformer_gpt_embedding('openai-gpt', test_x_list[2000:],TFGPT2Model)

In [ ]:
embedding_features.shape

In [ ]:
test_gpt_2_x = embedding_features[:,0,:]

In [ ]:
test_gpt_2_x.shape

In [ ]:
test_gpt_2_x_df = pd.DataFrame(test_gpt_2_x)

In [ ]:
test_gpt_2_x_df.head()

In [ ]:
test_gpt_2_x_df['is_sarcastic'] = test_y_list[2000:]

In [ ]:
test_gpt_2_x_df.head()

In [ ]:
test_gpt_2_x_df.to_csv('datasets/test_gpt_2_2000_end.csv', encoding='utf-8', index=False)

time: 2.24 s (started: 2021-07-23 19:59:12 +00:00)


# Merge Trainset


## Train

In [ ]:
train_gpt_2_1 = pd.read_csv("datasets/train_gpt_2_0_2000.csv") 
train_gpt_2_2 = pd.read_csv("datasets/train_gpt_2_2000_4000.csv") 
train_gpt_2_3 = pd.read_csv("datasets/train_gpt_2_4000_6000.csv") 
train_gpt_2_4 = pd.read_csv("datasets/train_gpt_2_6000_8000.csv") 
train_gpt_2_5 = pd.read_csv("datasets/train_gpt_2_8000_10000.csv") 
train_gpt_2_6 = pd.read_csv("datasets/train_gpt_2_10000_12000.csv") 
train_gpt_2_7 = pd.read_csv("datasets/train_gpt_2_12000_14000.csv") 
train_gpt_2_8 = pd.read_csv("datasets/train_gpt_2_14000_16000.csv")
train_gpt_2_9 = pd.read_csv("datasets/train_gpt_2_16000_18000.csv")
train_gpt_2_10 = pd.read_csv("datasets/train_gpt_2_18000_20000.csv") 
train_gpt_2_11 = pd.read_csv("datasets/train_gpt_2_20000_end.csv") 

time: 4.1 s (started: 2021-07-23 20:12:03 +00:00)


In [ ]:
train_gpt_2_df = train_gpt_2_1.append([train_gpt_2_2, train_gpt_2_3, train_gpt_2_4, train_gpt_2_5, train_gpt_2_6, train_gpt_2_7, train_gpt_2_8, train_gpt_2_9, train_gpt_2_10, train_gpt_2_11])

time: 35.3 ms (started: 2021-07-23 20:12:08 +00:00)


In [ ]:
train_gpt_2_df.shape

(22702, 769)

time: 3.49 ms (started: 2021-07-23 20:12:09 +00:00)


In [ ]:
train_gpt_2_df.to_csv('datasets/train_gpt_2.csv', encoding='utf-8', index=False)

time: 26.9 s (started: 2021-07-23 20:12:13 +00:00)


## Test

In [ ]:
test_gpt_2_1 = pd.read_csv("datasets/test_gpt_2_0_2000.csv") 
test_gpt_2_2 = pd.read_csv("datasets/test_gpt_2_2000_end.csv") 

time: 772 ms (started: 2021-07-23 20:14:25 +00:00)


In [ ]:
test_gpt_2_df = test_gpt_2_1.append([test_gpt_2_2])

time: 9.87 ms (started: 2021-07-23 20:14:29 +00:00)


In [ ]:
test_gpt_2_df.shape

(4007, 769)

time: 5.81 ms (started: 2021-07-23 20:14:35 +00:00)


In [ ]:
test_gpt_2_df.to_csv('datasets/test_gpt_2.csv', encoding='utf-8', index=False)

time: 4.89 s (started: 2021-07-23 20:14:46 +00:00)


# Statistical Models

In [ ]:
train_gpt_2_df = pd.read_csv("datasets/train_gpt_2.csv") 
train_gpt_2_df.shape 

(22702, 769)

time: 8.4 s (started: 2021-07-24 05:19:07 +00:00)


In [ ]:
train_gpt_2_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,is_sarcastic
0,0.180391,0.154627,0.102255,0.192836,0.284427,0.168536,-0.090936,-0.198666,-0.089844,0.512565,0.008530,-0.020906,-0.001444,-0.120649,-0.222980,-0.019511,0.393966,0.391520,0.313730,0.024529,-0.213853,0.203891,0.352329,0.118166,0.145137,0.509628,-0.273461,0.344705,-0.019176,0.054510,-0.004955,-0.230941,0.146816,0.171807,0.202423,-0.326025,0.328783,0.093912,0.065543,0.501761,...,0.046793,-0.081008,-0.062659,0.463721,0.005259,0.200312,0.773062,0.137061,0.113401,-0.017022,0.024501,-0.245807,0.369110,-0.229355,0.298966,0.206996,0.025464,-0.137082,0.131651,-0.038837,0.088767,0.124907,0.508232,0.149385,0.040577,0.129483,-0.201123,-0.081749,-0.038540,0.092638,0.292173,-0.334829,-0.125249,0.088591,0.213722,-0.113162,-0.227148,-0.009748,0.034562,0
1,0.055346,0.057804,-0.007890,0.292850,0.055851,0.277785,0.013854,-0.290726,-0.068341,0.293782,0.075334,-0.045700,0.019327,-0.149169,-0.020522,0.036281,0.321066,0.439734,0.160935,0.017454,-0.282512,0.212338,0.292898,0.206096,0.184880,0.388437,-0.295839,0.335922,-0.114147,0.101631,0.095916,-0.152639,0.210292,0.056946,0.021620,-0.302929,0.287128,0.083471,-0.147864,0.465112,...,0.040286,-0.205422,-0.176569,0.454670,0.145828,0.168703,1.265966,0.030530,0.036967,0.036569,0.048936,-0.148669,0.398008,-0.202223,0.348172,0.194118,0.018813,-0.071174,0.286279,-0.106020,0.133832,-0.003241,0.604737,0.062532,0.049915,0.231095,-0.144527,-0.097416,0.021033,0.146901,0.391437,-0.333802,-0.064749,0.102082,0.140693,-0.084642,0.063693,0.004927,0.069538,1
2,0.195907,-0.215209,0.234511,0.452022,0.069675,0.490711,-0.358129,-0.310998,0.146854,0.129032,-0.075022,0.047518,-0.133380,0.090660,-0.061415,0.276620,0.123244,0.703875,0.148393,0.121867,-0.173916,0.409447,0.529036,0.069816,0.243238,0.256276,-0.292166,0.256361,-0.137517,-0.165159,0.152264,-0.068725,0.147659,-0.234309,0.138459,-0.250781,0.044381,-0.042779,-0.066122,0.502444,...,0.235363,-0.385255,-0.057508,0.264337,-0.039426,0.059405,1.128448,0.076246,-0.187798,0.126881,0.098360,-0.176407,0.318862,-0.285548,0.260593,0.292177,-0.022576,0.172690,0.030060,0.071160,0.431645,0.226935,0.830682,-0.084265,0.001391,0.114945,0.026103,0.152788,-0.236838,0.001399,0.128591,-0.630190,-0.135552,0.038884,0.184727,-0.077423,-0.332696,-0.019556,0.218400,0
3,0.098043,0.133683,-0.001626,0.343836,0.332720,0.249848,-0.223203,-0.370426,-0.026909,0.570297,-0.116429,0.043016,-0.065494,-0.248303,-0.351312,0.096400,0.244159,0.387191,0.232075,0.097487,-0.170171,0.211687,0.309166,0.210909,0.175526,0.480872,-0.338139,0.354274,0.027733,0.158087,0.087651,-0.267137,0.157449,0.211512,0.044352,-0.401082,0.167519,0.087628,-0.127391,0.608151,...,-0.095448,-0.284538,-0.049697,0.567234,0.003263,0.297473,1.358630,0.026570,0.130837,0.185717,0.038375,-0.091977,0.292448,-0.207888,0.416913,0.279730,-0.015571,-0.193741,0.118354,-0.070762,0.045668,0.117517,0.659974,-0.018498,-0.034730,0.215064,-0.191103,0.059453,-0.041055,0.180616,0.287341,-0.427994,-0.201577,0.025986,0.035027,-0.037532,-0.397244,0.003961,0.228541,1
4,0.240887,0.041067,0.034774,0.252403,0.075464,0.338718,-0.142425,-0.350006,-0.006602,0.304148,0.067966,-0.035891,-0.017473,-0.163196,-0.179331,0.034581,0.369163,0.422605,0.238388,-0.006793,-0.155336,0.277875,0.399128,0.248042,0.275722,0.355250,-0.390046,0.283207,-0.173510,0.216597,0.073345,-0.226285,0.229761,-0.029256,0.172489,-0.231362,0.105001,-0.075090,0.058473,0.434520,...,0.164391,-0.180939,-0.144629,0.361130,0.079393,0.119582,1.468460,-0.030682,0.131370,0.152188,-0.037087,-0.313085,0.350249,-0.229776,0.330866,0.229785,-0.045805,-0.157083,0.196272,-0.208906,0.191593,0.068170,0.712308,0.035422,-0.056956,0.265626,-0.101805,-0.041143,-0.021304,0.124140,0.404433,-0.473180,-0.120533,0.121803,0.171305,-0.098672,-0.25404

time: 122 ms (started: 2021-07-24 05:19:16 +00:00)


In [ ]:
test_gpt_2_df = pd.read_csv("datasets/test_gpt_2.csv") 
test_gpt_2_df.shape 

(4007, 769)

time: 2.65 s (started: 2021-07-24 05:19:16 +00:00)


In [ ]:
train_y = train_gpt_2_df.pop('is_sarcastic')
test_y = test_gpt_2_df.pop('is_sarcastic')

train_x = train_gpt_2_df
test_x = test_gpt_2_df

train_x.shape,train_y.shape,test_x.shape,test_y.shape

((22702, 768), (22702,), (4007, 768), (4007,))

time: 10.4 ms (started: 2021-07-24 05:19:18 +00:00)


## SVC

In [ ]:
from sklearn.svm import SVC
svc = SVC(kernel ='rbf', gamma = 0.1, C = 10, probability=True) #2h 56min
svc.fit(train_x,train_y)
pred=svc.predict(test_x)

time: 4h 30min 54s (started: 2021-07-24 05:20:38 +00:00)


In [ ]:
print("Evaluate confusion matrix for LR")
print(confusion_matrix(test_y,pred))
print(f"Accuracy Score for LR with C=1.0  ={accuracy_score(test_y,pred)}")

Evaluate confusion matrix for LR
[[1739  509]
 [1340  419]]
Accuracy Score for LR with C=1.0  =0.5385575243324182
time: 16.4 ms (started: 2021-07-24 09:51:33 +00:00)


## Logistic Regression

In [ ]:
# Applying Logistic Regression 
model=LogisticRegression(max_iter=10000)
model.fit(train_x,train_y)
pred=model.predict(test_x)
print("Evaluate confusion matrix for LR")
print(confusion_matrix(test_y,pred))
print(f"Accuracy Score for LR with C=1.0  ={accuracy_score(test_y,pred)}")

Evaluate confusion matrix for LR
[[2029  219]
 [1573  186]]
Accuracy Score for LR with C=1.0  =0.5527826303968056
time: 33.2 s (started: 2021-07-23 20:15:26 +00:00)


## KNN

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 25, weights = 'distance')
knn.fit(train_x,train_y)
pred=knn.predict(test_x)

time: 1min 10s (started: 2021-07-23 20:16:18 +00:00)


In [ ]:
print("Evaluate confusion matrix for LR")
print(confusion_matrix(test_y,pred))
print(f"Accuracy Score for LR with C=1.0  ={accuracy_score(test_y,pred)}")

Evaluate confusion matrix for LR
[[1684  564]
 [1277  482]]
Accuracy Score for LR with C=1.0  =0.5405540304467182
time: 10.4 ms (started: 2021-07-23 20:17:28 +00:00)


## Decision Tree

In [ ]:
dt = DecisionTreeClassifier(criterion ='entropy', max_depth= None, min_samples_leaf=1, min_samples_split=20,splitter="best")
dt.fit(train_x,train_y)
pred=dt.predict(test_x)

time: 1min 8s (started: 2021-07-23 20:17:28 +00:00)


In [ ]:
print("Evaluate confusion matrix for LR")
print(confusion_matrix(test_y,pred))
print(f"Accuracy Score for LR with C=1.0  ={accuracy_score(test_y,pred)}")

Evaluate confusion matrix for LR
[[1346  902]
 [1082  677]]
Accuracy Score for LR with C=1.0  =0.5048664836536062
time: 9.42 ms (started: 2021-07-23 20:18:37 +00:00)


## Random Forest

In [ ]:
rf = RandomForestClassifier(criterion='entropy',n_estimators = 100, max_depth = None, bootstrap = False)
rf.fit(train_x,train_y)
pred=rf.predict(test_x)

time: 4min 29s (started: 2021-07-23 20:18:37 +00:00)


In [ ]:
print("Evaluate confusion matrix for LR")
print(confusion_matrix(test_y,pred))
print(f"Accuracy Score for LR with C=1.0  ={accuracy_score(test_y,pred)}")

Evaluate confusion matrix for LR
[[1909  339]
 [1484  275]]
Accuracy Score for LR with C=1.0  =0.5450461692038931
time: 10.5 ms (started: 2021-07-23 20:23:07 +00:00)


## Naive Bayes

In [ ]:
gnb = GaussianNB()
gnb.fit(train_x,train_y)
pred=gnb.predict(test_x)

time: 330 ms (started: 2021-07-23 20:23:07 +00:00)


In [ ]:
print("Evaluate confusion matrix for LR")
print(confusion_matrix(test_y,pred))
print(f"Accuracy Score for LR with C=1.0  ={accuracy_score(test_y,pred)}")

Evaluate confusion matrix for LR
[[ 965 1283]
 [ 723 1036]]
Accuracy Score for LR with C=1.0  =0.4993760918392813
time: 10.4 ms (started: 2021-07-23 20:23:07 +00:00)
